# We will try to solve below problem using OpenAIGym environment.

Frozen Lake:
Winter is here, You and your friends are tossing around the frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake.
The water is mostly frozen, but there are a few holes where the ice has melted. If you stem in one of those holes, you will fall into the freezing water. T this time, there is an international frisbee shortage, so it's absolutely imperitive that navigate across the lake and retrive the disk. However the ice is slippry so you won't always move in the direction you intend. The surface is described using a grid like the following...  

S F F F

F H F H

F F F H 

H F F G


Here S represents the starting point, it is safe.
F Frozon serface which is also safe
H is Hole, if is not good
G is Goal

The agent can navigate Left, Right, Up and Down.
It recieves the reward as 1 if it reches the goal, else 0.
The episode ends when either the agent falls in Hole or reaches the Goal.


In [ ]:
# import all lib
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [ ]:
env = gym.make('FrozenLake-v0')

In [ ]:
# remember: Row count: size of state space in env
#           Col count : size of action space in env
# 
# This can be extracted as below

In [15]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

print("action_space_size : ", action_space_size)
print("state_space_size : ", state_space_size)

action_space_size :  4
state_space_size :  16


In [16]:
# will use above info to build the Q table
q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [36]:
# Now lets define all the required parameters
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1            # epsilone
max_exploration_rate = 1
min_exploration_rate = 0.001     # how big and how smalll our exploration rate can be
exploration_decay_rate = 0.001   # Rate at which exploration rate will decayb

In [37]:
rewards_all_episodes = []

# Q Learning Algo
for episode in range(num_episodes):
    # for each episode reset the env to initial state
    state = env.reset()
    
    # done var will keep trackof if the episode is finished
    done = False
    # start with no rewards in the begining of episode
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        
        #exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
            
        new_state, reward, done, info = env.step(action)
        
        
        # update Q table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward
        
        if done == True:
            break
            
        
    #Exploration rate decay
    exploration_rate = min_exploration_rate + \
            (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)

    rewards_all_episodes.append(rewards_current_episode)
        


In [38]:
len(rewards_all_episodes)

10000

In [39]:
# Caclulate and print the avg reward per thousand episode
reward_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000
print("************** avg reward per thousand episodes ******************\n")
for r in reward_per_thousand_episodes:
    print(count, " : " ,str(sum(r/1000)) )
    count += 1000
    
    
# print updated Q table
print("***********updated Q table*************\n")
print(q_table)

************** avg reward per thousand episodes ******************

1000  :  0.05300000000000004
2000  :  0.21400000000000016
3000  :  0.4090000000000003
4000  :  0.5750000000000004
5000  :  0.6870000000000005
6000  :  0.7080000000000005
7000  :  0.7260000000000005
8000  :  0.7290000000000005
9000  :  0.7380000000000005
10000  :  0.7350000000000005
***********updated Q table*************

[[0.55873722 0.50358233 0.4983035  0.49801981]
 [0.3294781  0.34024087 0.28407139 0.52050647]
 [0.4107869  0.39516142 0.41334536 0.48090305]
 [0.32236689 0.277568   0.31970068 0.46311452]
 [0.57641444 0.3665535  0.22876746 0.36207069]
 [0.         0.         0.         0.        ]
 [0.16960956 0.22371334 0.3197786  0.17289626]
 [0.         0.         0.         0.        ]
 [0.44213139 0.4120752  0.37426615 0.61903489]
 [0.44455267 0.69104752 0.37867433 0.44014913]
 [0.57666129 0.42558925 0.41015798 0.32243559]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.       

# Rendering

In [45]:
for episode in range(3):
    state = env.reset()
    done = False
    print("*********** EPISODE", episode+1, "**********\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        
        action = np.argmax(q_table[state, :])
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            
            if reward == 1:
                print("You reached the Goal. \n")
                time.sleep(3)
            else:
                print("You fall throught a hole. \n")
                time.sleep(3)
            clear_output(wait=True)
            break
            
        state = new_state
        
env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG
You reached the Goal. 

